In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [3]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-11-24 16:44:40--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-11-24 16:44:40--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?bmOE6S0CqcrguHbheLyBpLJWlxUVSIfZO2o_a1Inzoe-mStvZPQQz-P0OhDEAcEFGrLDDPhs2O3xKv7AP8Udvi9HPhlaLW1uIf5wEQHdOROE6

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [6]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [13]:
%%cu


#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include "device_functions.h"
#include <thrust/device_vector.h>
#include <stdio.h>
#include <math.h>

#define EPSILON 				 0.01
#define SIZE_INTERVAL			 100
#define START_INTERVAL_RESEARCH -10
#define STOP_INTERVAL_RESEARCH   10
#define THREADS_PER_BLOCK		 3

#define INTERVAL (STOP_INTERVAL_RESEARCH - START_INTERVAL_RESEARCH)



/* функция корни которой необходимо найти */
__device__ double CalculateFancshion(double x)
{
	return  tan(1.9*x) - 2.8*x;
}
typedef struct
{
	double x;
    bool input;
} CalcBody;


/*Реализация на СЗГ*/
void CPU_realization(double *x)
{
	double a, b, c;
	a = START_INTERVAL_RESEARCH;//задаём границы исчесления
	b = STOP_INTERVAL_RESEARCH;
	while (b - a > epsilon) // начинаем цикл с условием выхода когда разница между правой и левой границей меньше эпсилон
	{
		c = (a + b) / 2;//середина отрезка		
		if (CalculateFancshion(b) * fCalculateFancshion(c) < 0)// условия в какую сторону двигаться
			b = c;
		else
			a = c;
	}	
	*x = (a + b) / 2;
}
/*функция реализующая метод половинного деления*/
__global__ void GPU_realization(CalcBody* body_structure, double dx, double start)
{
	__shared__ double flag_temp[3];
	__shared__ double value[3];
	value[threadIdx.x] = (blockIdx.x * dx + threadIdx.x * dx * 0.5) + start;
	if (GPU_realization(value[threadIdx.x]) == 0)
		body_structure[blockIdx.x].x = 0.;
	__syncthreads();
	while (true) 
	{
		if (GPU_realization(value[threadIdx.x]) < 0)
			flag_temp[threadIdx.x] = 0;
		else
			flag_temp[threadIdx.x] = 1;
		__syncthreads();
		if(flag_temp[threadIdx.x] == 1)
		{
			if (abs(value[threadIdx.x + 1] - value[threadIdx.x - 1]) < EPSILON)
			{
				body_structure[blockIdx.x].x = value[threadIdx.x];
                body_structure[blockIdx.x].input = true;
				break;
			}
			if (flag_temp[threadIdx.x - 1] != flag_temp[threadIdx.x])
			{
				value[threadIdx.x + 1] = value[threadIdx.x];
				value[threadIdx.x] = (value[threadIdx.x - 1] + value[threadIdx.x + 1]) / 2;
			}
			else if (flag_temp[threadIdx.x + 1] != flag_temp[threadIdx.x])
			{
				value[threadIdx.x - 1] = value[threadIdx.x];
				value[threadIdx.x] = (value[threadIdx.x - 1] + value[threadIdx.x + 1]) / 2;
 			}
		
 		}
 	}
 }


int main()
{
	setlocale(LC_ALL, "ru");
	thrust::device_vector<CalcBody>  vector(SIZE_INTERVAL);
	double dx = INTERVAL / SIZE_INTERVAL;

	cudaEvent_t start, stop;
	float gpu_time = 0.0;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);

	Calc_metod_polovin << <SIZE_INTERVAL, THREADS_PER_BLOCK >> > (thrust::raw_pointer_cast(& vector[0]), dx, START_INTERVAL_RESEARCH);

	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);

	cudaEventDestroy(start);
	cudaEventDestroy(stop);
	printf("\ntime = %2fmiliseconds;", gpu_time);
	printf("Корни уравнения ровняются:\n")
	for (int i = 0; i < SIZE_INTERVAL; i++)
	{
		CalcBody body = vector[i];
		if (body.input)
		{
			printf("x [%d] = %f\n", i, r.val);
		}
	}
 

	return 0;
}


time = 0.004520miliseconds;Корни уравнения ровняются:
x[23] = -0.499300
x[49] = 0.001300
x[76] = 0.500400

